<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# TL;DR

1. In this lab scenario you will have a chance to compare performance of the classic RNN and LSTM on a toy example.
2. This toy example will show that maintaining memory over even 20 steps is non-trivial.
3. Finally, you will see how curriculum learning may allow to train a model on larger sequences.

# Problem definition

Here we consider a toy example, where the goal is to discriminate between two types of binary sequences:
* [Type 0] a sequence with exactly one zero (remaining entries are equal to one).
* [Type 1] a sequence full of ones,

We are especially interested in the performance of the trained models on discriminating between a sequence full of ones versus a sequence with leading zero followed by ones. Note that in this case the goal of the model is to output the first element of the sequence, as the label (sequence type) is fully determined by the first element of the sequence.

#Implementation

## Importing torch

Install `torch` and `torchvision`

In [ ]:
!pip3 install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from typing import List
from tqdm import tqdm

torch.manual_seed(1)

## Understand dimensionality

Check the input and output specification [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) and [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). The following snippet shows how we can process
a sequence by LSTM and output a vector of size `hidden_dim` after reading
each token of the sequence.

In [ ]:
hidden_dim = 5
lstm = nn.LSTM(1, hidden_dim)  # Input sequence contains elements - vectors of size 1

# create a random sequence
sequence = [torch.randn(1) for _ in range(10)]

# initialize the hidden state (including cell state)
hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))

for i, elem in enumerate(sequence):
    # we are processing only a single element of the sequence, and there
    # is only one sample (sequence) in the batch, the third one
    # corresponds to the fact that our sequence contains elemenents,
    # which can be treated as vectors of size 1
    out, hidden = lstm(elem.view(1, 1, 1), hidden)
    print(f'i={i} out={out.detach()}')
print(f'Final hidden state={hidden[0].detach()} cell state={hidden[1].detach()}')

i=0 out=tensor([[[-0.0675,  0.1179,  0.1081,  0.0414, -0.0341]]])
i=1 out=tensor([[[-0.1067,  0.1726,  0.1400,  0.0902, -0.0596]]])
i=2 out=tensor([[[-0.1148,  0.1885,  0.1956,  0.0974, -0.0840]]])
i=3 out=tensor([[[-0.1270,  0.2031,  0.1495,  0.1249, -0.0860]]])
i=4 out=tensor([[[-0.1281,  0.2019,  0.1810,  0.1475, -0.1027]]])
i=5 out=tensor([[[-0.1274,  0.2060,  0.0798,  0.1330, -0.0860]]])
i=6 out=tensor([[[-0.1318,  0.2039,  0.0997,  0.1772, -0.1011]]])
i=7 out=tensor([[[-0.1145,  0.2008, -0.0431,  0.1051, -0.0717]]])
i=8 out=tensor([[[-0.1289,  0.1989,  0.0515,  0.1944, -0.1030]]])
i=9 out=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]])
Final hidden state=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]]) cell state=tensor([[[-0.2590,  0.4080,  0.1307,  0.4329, -0.2895]]])


## To implement

Process the whole sequence all at once by calling `lstm` only once and check that the output is exactly the same as above (remember to initialize the hidden state the same way).

In [ ]:
input = torch.cat(sequence)

hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))

out, hidden = lstm(input.view(-1, 1, 1), hidden)
print(out.detach())

tensor([[[-0.0675,  0.1179,  0.1081,  0.0414, -0.0341]],

        [[-0.1067,  0.1726,  0.1400,  0.0902, -0.0596]],

        [[-0.1148,  0.1885,  0.1956,  0.0974, -0.0840]],

        [[-0.1270,  0.2031,  0.1495,  0.1249, -0.0860]],

        [[-0.1281,  0.2019,  0.1810,  0.1475, -0.1027]],

        [[-0.1274,  0.2060,  0.0798,  0.1330, -0.0860]],

        [[-0.1318,  0.2039,  0.0997,  0.1772, -0.1011]],

        [[-0.1145,  0.2008, -0.0431,  0.1051, -0.0717]],

        [[-0.1289,  0.1989,  0.0515,  0.1944, -0.1030]],

        [[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]])


## Training a model

Below we define a very simple model, which is a single layer of LSTM, where the output in each time step is processed by relu followed by a single fully connected layer, the output of which is a single number. We are going
to use the number generated after reading the last element of the sequence,
which will serve as the logit for our classification problem.

In [ ]:
class Model(nn.Module):
    def __init__(self, hidden_dim: int):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.lstm(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

Below is a training loop, where we only train on the two hardest examples.

In [ ]:
SEQUENCE_LEN = 10

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]


def eval_on_hard_examples(model: nn.Module) -> List[float]:
    with torch.no_grad():
        logits = []
        for sequence in HARD_EXAMPLES:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')
        return logits


def train_model(hidden_dim: int, lr: float, num_steps: int = 10000):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    pbar = tqdm(range(num_steps))
    for step in pbar:
        if step % 100 == 0:
            logits = eval_on_hard_examples(model)
            pbar.set_postfix(logits=logits)

        for sequence, label in HARD_EXAMPLES:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))

            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()

In [ ]:
train_model(hidden_dim=20, lr=0.01, num_steps=10000)

  0%|          | 29/10000 [00:00<01:06, 150.43it/s, logits=[tensor([0.0646]), tensor([0.0645])]]

Logits for hard examples=[tensor([0.0646]), tensor([0.0645])]


  1%|          | 122/10000 [00:00<00:57, 171.28it/s, logits=[tensor([0.0233]), tensor([0.0233])]]

Logits for hard examples=[tensor([0.0233]), tensor([0.0233])]


  2%|▏         | 241/10000 [00:01<00:50, 194.94it/s, logits=[tensor([-0.0064]), tensor([-0.0064])]]

...

 98%|█████████▊| 9834/10000 [00:37<00:00, 281.63it/s, logits=[tensor([-9.0098]), tensor([8.6415])]]

Logits for hard examples=[tensor([-9.0098]), tensor([8.6415])]


100%|█████████▉| 9953/10000 [00:37<00:00, 292.37it/s, logits=[tensor([-9.0302]), tensor([8.6623])]]

Logits for hard examples=[tensor([-9.0302]), tensor([8.6623])]


100%|██████████| 10000/10000 [00:37<00:00, 264.67it/s, logits=[tensor([-9.0302]), tensor([8.6623])]]


## To implement

1. Check for what values of `SEQUENCE_LEN` the model is able to discriminate betweeh the two hard examples (after training).
2. Instead of training on `HARD_EXAMPLES` only, modify the training loop to train on sequences where zero may be in any position of the sequence (so any valid sequence of `Type 0`, not just the hardest one). After modifying the training loop check for what values of `SEQUENCE_LEN` you can train the model successfully.
3. Replace LSTM by a classic RNN and check for what values of `SEQUENCE_LEN` you can train the model successfully.
4. Write a proper curricullum learning loop, where in a loop you consider longer and longer sequences, where expansion of the sequence length happens only after the model is trained successfully on the current length.

Note that for steps 2-4 you may need to change the value of `num_steps`.

In [ ]:
SEQUENCE_LEN = 20
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]
train_model(hidden_dim=20, lr=0.01, num_steps=10000)

  1%|          | 58/10000 [00:00<00:34, 286.21it/s, logits=[tensor([-0.0578]), tensor([-0.0578])]]

Logits for hard examples=[tensor([-0.0578]), tensor([-0.0578])]


  1%|▏         | 146/10000 [00:00<00:34, 285.25it/s, logits=[tensor([0.0005]), tensor([0.0005])]]

Logits for hard examples=[tensor([0.0005]), tensor([0.0005])]


  2%|▏         | 234/10000 [00:00<00:36, 265.32it/s, logits=[tensor([0.0115]), tensor([0.0115])]]

...

 99%|█████████▊| 9859/10000 [00:41<00:00, 275.66it/s, logits=[tensor([0.0013]), tensor([0.0013])]]

Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]


100%|█████████▉| 9950/10000 [00:42<00:00, 276.58it/s, logits=[tensor([0.0013]), tensor([0.0013])]]

Logits for hard examples=[tensor([0.0013]), tensor([0.0013])]


100%|██████████| 10000/10000 [00:42<00:00, 236.28it/s, logits=[tensor([0.0013]), tensor([0.0013])]]


In [ ]:
SEQUENCE_LEN = 10
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]
EXAMPLES = [(SEQUENCE_LEN*[1.], 1)]
for i in range(SEQUENCE_LEN):
    EXAMPLES.append((i*[1.]+[0.]+(SEQUENCE_LEN-1-i)*[1.], 0))

def train_model(hidden_dim: int, lr: float, num_steps: int = 10000):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    pbar = tqdm(range(num_steps))
    for step in pbar:
        if step % 100 == 0:
            logits = eval_on_hard_examples(model)
            pbar.set_postfix(logits=logits)

        for sequence, label in EXAMPLES:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))

            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()

train_model(hidden_dim=20, lr=0.01, num_steps=1000)

  1%|          | 6/1000 [00:00<00:18, 53.34it/s, logits=[tensor([-0.2119]), tensor([-0.2118])]]

Logits for hard examples=[tensor([-0.2119]), tensor([-0.2118])]


 11%|█         | 108/1000 [00:01<00:16, 54.04it/s, logits=[tensor([-2.0934]), tensor([-2.0924])]]

Logits for hard examples=[tensor([-2.0934]), tensor([-2.0924])]


 21%|██        | 210/1000 [00:03<00:14, 55.81it/s, logits=[tensor([-6.8077]), tensor([4.2476])]]

...

 81%|████████  | 811/1000 [00:15<00:03, 54.18it/s, logits=[tensor([-10.5136]), tensor([7.4703])]]

Logits for hard examples=[tensor([-10.5136]), tensor([7.4703])]


 91%|█████████ | 907/1000 [00:17<00:01, 54.75it/s, logits=[tensor([-10.7088]), tensor([7.5540])]]

Logits for hard examples=[tensor([-10.7088]), tensor([7.5540])]


100%|██████████| 1000/1000 [00:19<00:00, 50.74it/s, logits=[tensor([-10.7088]), tensor([7.5540])]]


In [ ]:
SEQUENCE_LEN = 6
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]
EXAMPLES = [(SEQUENCE_LEN*[1.], 1)]
for i in range(SEQUENCE_LEN):
    EXAMPLES.append((i*[1.]+[0.]+(SEQUENCE_LEN-1-i)*[1.], 0))

class Model(nn.Module):
    def __init__(self, hidden_dim: int):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.rnn(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

train_model(hidden_dim=20, lr=0.001, num_steps=2000)

  0%|          | 7/2000 [00:00<00:31, 63.65it/s, logits=[tensor([0.0076]), tensor([0.0054])]]

Logits for hard examples=[tensor([0.0076]), tensor([0.0054])]


  5%|▌         | 109/2000 [00:01<00:31, 59.31it/s, logits=[tensor([-1.7686]), tensor([-1.7685])]]

Logits for hard examples=[tensor([-1.7686]), tensor([-1.7685])]


 10%|█         | 210/2000 [00:03<00:22, 80.16it/s, logits=[tensor([-1.7142]), tensor([-1.7130])]]

...

 91%|█████████ | 1814/2000 [00:22<00:02, 88.24it/s, logits=[tensor([-10.9042]), tensor([4.8795])]]

Logits for hard examples=[tensor([-10.9042]), tensor([4.8795])]


 96%|█████████▌| 1914/2000 [00:23<00:01, 81.17it/s, logits=[tensor([-11.0520]), tensor([4.9670])]]

Logits for hard examples=[tensor([-11.0520]), tensor([4.9670])]


100%|██████████| 2000/2000 [00:24<00:00, 80.73it/s, logits=[tensor([-11.0520]), tensor([4.9670])]]
